In [ ]:
##### import required modules #####
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys

##### import public transport data #####
public_transport_df = pd.read_csv('public_transport.csv')
# set index to start at 1
public_transport_df.index += 1

In [ ]:
##### define functions #####

## set_up ##
# opens and sets up the google maps page
# arrive_time: set the arrive time; format 'hh:mm'
# week_day: set the arrive day; accepted value = True (day is set to monday), False (day is set to saturday)

def set_up(arrive_time,week_day):

    # assign url in the webdriver object
    global driver
    driver = webdriver.Chrome()
    driver.get("https://www.google.it/maps/dir/@45.5038144,9.4419926,15z?entry=ttu")
    sleep(2)

    # Accept cookie policy
    Submit = driver.find_elements(By.TAG_NAME, "button")
    Submit[1].click()
    sleep(2)

    # initialize the page with an arbitrary search
    searchplace("metro precotto","Piazza della Scienza, 20126 Milano MI")
    sleep(5)

    # select "arrive by" mode
    driver.find_element(By.XPATH,'//*[@id=":3"]').click()
    driver.find_element(By.XPATH,'//*[@id=":2"]').click()
    sleep(1)

    # select arrive time
    path ='//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/span[1]/input'
    driver.find_element(By.XPATH,path).clear()
    driver.find_element(By.XPATH,path).send_keys(arrive_time)
    sleep(1)

    # select arrive day
    path_testo ='//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/span[2]/span[1]'
    path_bottone = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div/span[2]/span[2]/button[2]'
    giorno = driver.find_element(By.XPATH,path_testo).text

    if week_day:
        target = 'lun'
    else:
        target = 'sab'

    while (not target in giorno):
        driver.find_element(By.XPATH,path_bottone).click()
        giorno = driver.find_element(By.XPATH,path_testo).text
        sleep(1)


## searchplace ##
# inputs starting point (A) and destination (B)

def searchplace(A,B):
    Place = driver.find_elements(By.CLASS_NAME,"tactile-searchbox-input")
    Place[0].clear()
    Place[0].send_keys(A)
    Place[1].clear()
    Place[1].send_keys(B)
    Place[1].send_keys(Keys.RETURN)


## extract_time ##
# extracts the time needed to get from A to B by public transport

def extract_time():
    # read time from the web page
    path_tempo = '//*[@id="omnibox-directions"]/div/div[2]/div/div/div/div[3]/button/div[1]'
    tempo = driver.find_element(By.XPATH,path_tempo).text
    # convert time format in number of minutes
    if tempo!='':
        numbers = [int(s) for s in tempo.split() if s.isdigit()]
        if len(numbers)==2:
            time = numbers[0]*60 + numbers[1]
        else:
            time = numbers[0]
        return int(time)


## time_table ##
# compute the time needed to get from each metro and train station to a given point of interest
# point of interest: a string that google maps interprets as the desired place
# delay: sets the timing of the loop, adjust for internet speed
# arrive_time: set the arrive time; format 'hh:mm'
# week_day: set the arrive day; accepted value = True(day is set to monday), False(day is set to saturday)

def time_table(point_of_intrest, arrive_time, week_day ,delay=5):
    set_up(arrive_time,week_day)
    time_list = []
    station_id = []

    # loops over stations and retrives the needed time storing it along
    # with the station index in time_list an station_id respectively
    for index,row in tqdm(public_transport_df.iterrows(),total=public_transport_df.shape[0], desc='Progress'):
        station = row['name'] + ' ' + row['station_type']
        searchplace(station,point_of_intrest)
        sleep(delay)
        tempo = extract_time()
        time_list.append(tempo)
        station_id.append(index)

    # creates a dataframe containing station_id and time_list
    df = pd.DataFrame(data={'station_id': station_id, 'time': time_list})
    return df

In [ ]:
# exemple
test_df2 = time_table("Piazza della Scienza, 20126 Milano MI", arrive_time='8:30', week_day=True, delay=3)
test_df2.to_csv('test_df.csv', index=False)

Progress:  30%|██▉       | 55/184 [03:20<07:50,  3.65s/it]


KeyboardInterrupt: 